In [23]:
from torchPIV import OfflinePIV
import torch

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import cv2
import os

In [2]:
gpu_enabled = torch.cuda.is_available()

if gpu_enabled:
    device = torch.cuda.get_device_name()
else:
    device = 'cpu'

In [30]:
piv_gen = OfflinePIV(
    folder="test_imgs", # Path to experiment
    device=device, # Device name
    file_fmt="bmp",
    wind_size=32,
    overlap=16,
    dt=12, # Time between frames, mcs
    scale = 0.02, # mm/pix
    multipass=1,
    multipass_mode="CWS", # CWS or DWS
    multipass_scale=2.0, # Window downscale on each pass
    folder_mode="pairs" # Pairs or sequential frames 
)

results = []
for out in piv_gen():
    x, y, vx, vy = out
    results.append(out)

Load time 0.035 sec Batch finished in 0.736 sec
Load time 0.017 sec Batch finished in 0.403 sec
Load time 0.016 sec Batch finished in 0.387 sec
Load time 0.018 sec Batch finished in 0.378 sec
Load time 0.017 sec Batch finished in 0.372 sec
Load time 0.017 sec Batch finished in 0.411 sec
Load time 0.016 sec Batch finished in 0.336 sec
Load time 0.017 sec Batch finished in 0.371 sec
Load time 0.017 sec Batch finished in 0.349 sec
Load time 0.016 sec Batch finished in 0.356 sec
Load time 0.016 sec Batch finished in 0.358 sec


In [32]:
files = os.listdir(r'test_imgs')

velocities = []

for _, _, vx, vy in results:
    abs_velocity = np.sqrt(vx ** 2 + vy ** 2)
    velocities.append(abs_velocity)
    
velocities = np.array(velocities)
velocities /= np.max(velocities)

for idx, (x, y, vx, vy) in enumerate(results):
    img = cv2.imread(rf'test_imgs/{files[idx*2]}', cv2.IMREAD_GRAYSCALE)

    new_x = x/piv_gen._scale
    new_y = y/piv_gen._scale

    abs_velocity = np.sqrt(vx ** 2 + vy ** 2)
    abs_velocity /= np.max(abs_velocity)

    plt.imshow(img)
    plt.quiver(new_x, new_y, vx, vy, velocities[idx], cmap='jet')
    plt.show()

In [33]:
for img_file in os.listdir(r'test_imgs'):
    img = cv2.imread(rf'test_imgs/{img_file}', cv2.IMREAD_GRAYSCALE)
    xsize, ysize = img.shape
    
    newx, newy = int(xsize*0.3), int(ysize*0.3)
    img = cv2.resize(img, (newx, newy), interpolation= cv2.INTER_LINEAR)
    
    cv2.imshow(str(img_file), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()